In [49]:
import os
import pandas as pd
import numpy as np
import json
import pickle
from pprint import pprint
from collections import defaultdict
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

import torch

import os, sys
parentPath = os.path.abspath("..")
if parentPath not in sys.path:
    sys.path.insert(0, parentPath)
    
from src.data import load_source, slice_d
from src.config import Config, get_option_fallback
from src.path import get_best_model_paths, get_exp_paths, get_report_path, load_json, load_rep_cfg
from knowledge_tracing.trainer import Trainer
pd.options.display.max_columns = None

In [2]:
projectdir = Path('/code')
assert projectdir.exists()

In [40]:
with open(projectdir / 'data/raw_input/ds507_tx_2020_0223_005048/ds507_tx_All_Data_1664_2017_0227_034415.txt', 'r') as f:
    tx_df = pd.read_table(f)

/root/.pyenv/versions/3.7.4/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [120]:
with open(projectdir / 'data/raw_input/ds507_tx_2020_0223_005048/ds507_tx_cmu_muohio_1672_2017_0227_034923.txt', 'r') as f:
    cmu_df = pd.read_table(f)

In [22]:
with open(projectdir / 'data/raw_input/ds507_student_step_2020_0223_005452/ds507_student_step_All_Data_1664_2017_0227_035544.txt', 'r') as f:
    stustep_df = pd.read_table(f)

In [50]:
tx_df[tx_df['Student Response Type'] == 'ATTEMPT'][tx_df['Outcome'].notnull()]

/root/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Row,Sample Name,Transaction Id,Anon Student Id,Session Id,Time,Time Zone,Duration (sec),Student Response Type,Student Response Subtype,Tutor Response Type,Tutor Response Subtype,Level (Sequence),Level (Unit),Level (Module),Level (Section1),Problem Name,Problem View,Problem Start Time,Step Name,Attempt At Step,Is Last Attempt,Outcome,Selection,Action,Input,Input.1,Feedback Text,Feedback Classification,Help Level,Total Num Hints,KC (Single-KC),KC Category (Single-KC),KC (Unique-step),KC Category (Unique-step),KC (F2011),KC Category (F2011),KC (F2011).1,KC Category (F2011).1,KC (F2011).2,KC Category (F2011).2,School,Class,CF (oli:activityGuid),CF (oli:highStakes),CF (oli:purpose),CF (oli:resourceType)
1,2,All Data,4393251e32a6f00502f3f1ef894af8fe,Stu_00b2b35fd027e7891e8a1a527125dd65,8dd109e680020ca6016f8e64290b5610,2011-09-21 17:35:28,US/Eastern,23.13,ATTEMPT,NaN,RESULT,NaN,Statics,Concentrated Forces and Their Effects,Introduction to Free Body Diagrams,NaN,_m2_assess,1,2011-09-21 17:26:35,q1_point1i1 UpdateComboBox,1.0,1.0,CORRECT,q1_point1i1,UpdateComboBox,<material>cord c</material>,NaN,NaN,NaN,NaN,NaN,Single-KC,NaN,NaN,NaN,identify_interaction,NaN,NaN,NaN,NaN,NaN,Marion Technical College,MET2010B-01,NaN,NaN,NaN,NaN
2,3,All Data,e2fb2cb788d10ebaa6f288e0757d1b09,Stu_00b2b35fd027e7891e8a1a527125dd65,8dd109e680020ca6016f8e64290b5610,2011-09-21 17:35:28,US/Eastern,23.13,ATTEMPT,NaN,RESULT,NaN,Statics,Concentrated Forces and Their Effects,Introduction to Free Body Diagrams,NaN,_m2_assess,1,2011-09-21 17:26:35,q1_point3i3 UpdateComboBox,1.0,1.0,CORRECT,q1_point3i3,UpdateComboBox,<material>120 lb</material>,NaN,NaN,NaN,NaN,NaN,Single-KC,NaN,NaN,NaN,gravitational_forces,NaN,NaN,NaN,NaN,NaN,Marion Technical College,MET2010B-01,NaN,NaN,NaN,NaN
3,4,All Data,e7e150d423862e346dc7e36a95e394e4,Stu_00b2b35fd027e7891e8a1a527125dd65,8dd109e680020ca6016f8e64290b5610,2011-09-21 17:35:28,US/Eastern,23.13,ATTEMPT,NaN,RESULT,NaN,Statics,Concentrated Forces and Their Effects,Introduction to Free Body Diagrams,NaN,_m2_assess,1,2011-09-21 17:26:35,q1_point6i2 UpdateComboBox,1.0,1.0,INCORRECT,q1_point6i2,UpdateComboBox,<material>no interaction</material>,NaN,NaN,NaN,NaN,NaN,Single-KC,NaN,NaN,NaN,represent_interaction_spring,NaN,NaN,NaN,NaN,NaN,Marion Technical College,MET2010B-01,NaN,NaN,NaN,NaN
4,5,All Data,684b1f770a225f21745c6c4c977ddc32,Stu_00b2b35fd027e7891e8a1a527125dd65,8dd109e680020ca6016f8e64290b5610,2011-09-21 17:35:28,US/Eastern,23.13,ATTEMPT,NaN,RESULT,NaN,Statics,Concentrated Forces and Their Effects,Introduction to Free Body Diagrams,NaN,_m2_assess,1,2011-09-21 17:26:35,q1_point1i2 UpdateComboBox,1.0,1.0,CORRECT,q1_point1i2,UpdateComboBox,<material>up</material>,NaN,NaN,NaN,NaN,NaN,Single-KC,NaN,NaN,NaN,represent_interaction_cord,NaN,NaN,NaN,NaN,NaN,Marion Technical College,MET2010B-01,NaN,NaN,NaN,NaN
5,6,All Data,451a90f5c5e2711ecdef2b2e23145565,Stu_00b2b35fd027e7891e8a1a527125dd65,8dd109e680020ca6016f8e64290b5610,2011-09-21 17:35:28,US/Eastern,23.13,ATTEMPT,NaN,RESULT,NaN,Statics,Concentrated Forces and Their Effects,Introduction to Free Body Diagrams,NaN,_m2_assess,1,2011-09-21 17:26:35,q1_point4i1 UpdateComboBox,1.0,1.0,CORRECT,q1_point4i1,UpdateComboBox,<material>no body</material>,NaN,NaN,NaN,NaN,NaN,Single-KC,NaN,NaN,NaN,identify_interaction,NaN,NaN,NaN,NaN,NaN,Marion Technical College,MET2010B-01,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361081,361082,All Data,2c0a3adafa805db45be32f28532cf3d1,Stu_ff8a0b621620e0553bf9897e00a3b13a,57e7e49c80020ca600aefcb02a9736e1,2011-10-31 02:57:14,US/Eastern,5.8,ATTEMPT,NaN,RESULT,NaN,Statics,Friction,Friction,NaN,_m22_assess,1,2011-10-31 02:52:58,q1_i1 UpdateRadioButton,1.0,1.0,CORRECT,q1_i1,UpdateRadioButton,<material>9 N</material>,NaN,<material />,NaN,NaN,NaN,Single-KC,NaN,KC729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Carnegie Mellon Universi

In [28]:
stustep_df[[
#  'Row',
#  'Sample',
#  'Anon Student Id',
#  'Problem Hierarchy',
#  'Problem Name',
#  'Problem View',
#  'Step Name',
#  'Step Start Time',
#  'First Transaction Time',
#  'Correct Transaction Time',
#  'Step End Time',
#  'Step Duration (sec)',
#  'Correct Step Duration (sec)',
#  'Error Step Duration (sec)',
 'First Attempt',
 'Incorrects',
 'Hints',
 'Corrects',
 'Condition',
 'KC (F2011)',
 'Opportunity (F2011)',
 'Predicted Error Rate (F2011)',
 'KC (Single-KC)',
 'Opportunity (Single-KC)',
 'Predicted Error Rate (Single-KC)',
 'KC (Unique-step)',
 'Opportunity (Unique-step)',
 'Predicted Error Rate (Unique-step)'
   ]]

,First Transaction Time,Correct Transaction Time,Step End Time,Step Duration (sec),Correct Step Duration (sec),Error Step Duration (sec),First Attempt,Incorrects,Hints,Corrects,Condition,KC (F2011),Opportunity (F2011),Predicted Error Rate (F2011),KC (Single-KC),Opportunity (Single-KC),Predicted Error Rate (Single-KC),KC (Unique-step),Opportunity (Unique-step),Predicted Error Rate (Unique-step)
0,2011-09-21 17:35:28,2011-09-21 17:35:28,2011-09-21 17:35:28,23.13,23.13,.,correct,0,0,1,.,identify_interaction,1,0.3991,Single-KC,1,0.4373,NaN,NaN,NaN
1,2011-09-21 17:35:28,2011-09-21 17:35:28,2011-09-21 17:35:28,23.13,23.13,.,correct,0,0,1,.,gravitational_forces,1,0.1665,Single-KC,2,0.4373,NaN,NaN,NaN
2,2011-09-21 17:35:28,NaN,2011-09-21 17:35:28,23.13,.,23.13,incorrect,1,0,0,.,represent_interaction_spring,1,0.4120,Single-KC,3,0.4373,NaN,NaN,NaN
3,2011-09-21 17:35:28,2011-09-21 17:35:28,2011-09-21 17:35:28,23.13,23.13,.,correct,0,0,1,.,represent_interaction_cord,1,0.2619,Single-KC,4,0.4373,NaN,NaN,NaN
4,2011-09-21 17:35:28,2011-09-21 17:35:28,2011-09-21 17:35:28,23.13,23.13,.,correct,0,0,1,.,identify_interaction,2,0.3991,Single-KC,5,0.4373,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194942,2011-10-31 02:57:14,2011-10-31 02:57:14,2011-10-31 02:57:14,5.8,5.8,.,correct,0,0,1,.,NaN,NaN,NaN,Single-KC,512,0.2825,KC604,1.0,NaN
194943,2011-10-31 02:57:14,NaN,2011-10-31 02:57:14,5.8,.,5.8,incorrect,1,0,0,.,NaN,NaN,NaN,Single-KC,513,0.2825,KC966,1.0,NaN
194944,2011-10-31 02:57:14,2011-10-31 02:57:14,2011-10-31 02:57:14,5.8,5.8,.,correct,0,0,1,.,NaN,NaN,NaN,Single-KC,514,0.2825,KC1057,1.0,NaN
194945,2011-10-31 02:57:14,2011-10-31 02:57:14,2011-10-31 02:57:14,5.8,5.8,.,correct,0,0,1,.,NaN,NaN,NaN,Single-KC,515,0.2825,KC723,1.0,NaN


In [51]:
tmpdf = tx_df[[
#  'Row',
#  'Sample Name',
#  'Transaction Id',
 'Anon Student Id',  # student id
#  'Session Id',
#  'Time',
#  'Time Zone',
#  'Duration (sec)',
 'Student Response Type',
#  'Student Response Subtype',
#  'Tutor Response Type',
#  'Tutor Response Subtype',
#  'Level (Sequence)',
#  'Level (Unit)',
#  'Level (Module)',
#  'Level (Section1)',
#  'Problem Name',
#  'Problem View',
#  'Problem Start Time',
#  'Step Name',
#  'Attempt At Step',
#  'Is Last Attempt',
 'Outcome',  # is correct or not
#  'Selection',
#  'Action',
#  'Input',
#  'Input.1',
#  'Feedback Text',
#  'Feedback Classification',
#  'Help Level',
#  'Total Num Hints',
 'KC (Single-KC)',
 'KC Category (Single-KC)',
 'KC (Unique-step)',
 'KC Category (Unique-step)',
 'KC (F2011)',
 'KC Category (F2011)',
 'KC (F2011).1',
 'KC Category (F2011).1',
 'KC (F2011).2',
 'KC Category (F2011).2',
#  'School',
#  'Class',
#  'CF (oli:activityGuid)',
#  'CF (oli:highStakes)',
#  'CF (oli:purpose)',
#  'CF (oli:resourceType)'
   ]]

In [55]:
tmpdf[tmpdf['Student Response Type'] == 'ATTEMPT'][
   tmpdf[['KC (Single-KC)',
 'KC Category (Single-KC)',
 'KC (Unique-step)',
 'KC Category (Unique-step)',
 'KC (F2011)',
 'KC Category (F2011)',
 'KC (F2011).1',
 'KC Category (F2011).1',
 'KC (F2011).2',
 'KC Category (F2011).2']].notnull().any(1)
].size

/root/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()


3405311

In [58]:
tmpdf[tmpdf['Student Response Type'] == 'ATTEMPT']['Anon Student Id'].unique().size

335

In [114]:
for col in tx_df.columns:
    print(col, tx_df[col].nunique())

Row 361092
Sample Name 1
Transaction Id 361092
Anon Student Id 335
Session Id 6656
Time 263172
Time Zone 1
Duration (sec) 2565
Student Response Type 5
Student Response Subtype 1
Tutor Response Type 2
Tutor Response Subtype 0
Level (Sequence) 1
Level (Unit) 7
Level (Module) 19
Level (Section1) 9
Problem Name 300
Problem View 32
Problem Start Time 46473
Step Name 382
Attempt At Step 427
Is Last Attempt 2
Outcome 3
Selection 286
Action 9
Input 6826
Input.1 1
Feedback Text 1578
Feedback Classification 0
Help Level 0
Total Num Hints 0
KC (Single-KC) 1
KC Category (Single-KC) 0
KC (Unique-step) 1178
KC Category (Unique-step) 0
KC (F2011) 80
KC Category (F2011) 0
KC (F2011).1 18
KC Category (F2011).1 0
KC (F2011).2 8
KC Category (F2011).2 0
School 7
Class 9
CF (oli:activityGuid) 1243
CF (oli:highStakes) 2
CF (oli:purpose) 3
CF (oli:resourceType) 2


In [131]:
atmptdf = tx_df[tx_df['Outcome'].notnull()]
atmptdf\
[
#     atmptdf['KC (Single-KC)'].notnull() | 
    atmptdf['KC (Unique-step)'].notnull() #| 
#     atmptdf['CF (oli:activityGuid)'].notnull() #| 
#     atmptdf['KC (F2011)'].notnull() | 
#     atmptdf['KC (F2011).1'].notnull() | 
#     atmptdf['KC (F2011).2'].notnull()
]\
[atmptdf['Outcome'].notnull()]\
[atmptdf['Outcome'] != 'HINT']\
[atmptdf['Student Response Type'] == 'ATTEMPT']\
.shape

/root/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
/root/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':
/root/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]


(258113, 47)

In [130]:
atmptdf.Outcome.value_counts()

CORRECT      187943
INCORRECT     74004
HINT          27911
Name: Outcome, dtype: int64

In [64]:
atmptdf.notnull()

,Anon Student Id,Student Response Type,Outcome,KC (Single-KC),KC Category (Single-KC),KC (Unique-step),KC Category (Unique-step),KC (F2011),KC Category (F2011),KC (F2011).1,KC Category (F2011).1,KC (F2011).2,KC Category (F2011).2
1,True,True,True,True,False,False,False,True,False,False,False,False,False
2,True,True,True,True,False,False,False,True,False,False,False,False,False
3,True,True,True,True,False,False,False,True,False,False,False,False,False
4,True,True,True,True,False,False,False,True,False,False,False,False,False
5,True,True,True,True,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
361081,True,True,True,True,False,True,False,False,False,False,False,False,False
361082,True,True,True,True,False,True,False,False,False,False,False,False,False
361083,True,True,True,True,False,True,False,False,False,False,False,False,False
361084,True,True,True,True,False,True,False,False,False,False,False,False,False


# cmu

In [124]:
cmu_df[[
#     'Row',
#  'Sample Name',
#  'Transaction Id',
 'Anon Student Id',
#  'Session Id',
#  'Time',
#  'Time Zone',
#  'Duration (sec)',
 'Student Response Type',
 'Student Response Subtype',
 'Tutor Response Type',
 'Tutor Response Subtype',
 'Level (Sequence)',
 'Level (Unit)',
 'Level (Module)',
 'Level (Section1)',
 'Problem Name',
 'Problem View',
 'Problem Start Time',
 'Step Name',
 'Attempt At Step',
 'Is Last Attempt',
 'Outcome',
 'Selection',
 'Action',
 'Input',
 'Feedback Text',
 'Feedback Classification',
 'Help Level',
 'Total Num Hints',
 'KC (Single-KC)',
 'KC Category (Single-KC)',
 'KC (Unique-step)',
 'KC Category (Unique-step)',
 'KC (F2011)',
 'KC Category (F2011)',
 'KC (F2011).1',
 'KC Category (F2011).1',
 'KC (F2011).2',
 'KC Category (F2011).2',
 'School',
 'Class',
 'CF (oli:activityGuid)',
 'CF (oli:highStakes)',
 'CF (oli:purpose)',
 'CF (oli:resourceType)']]

['Row',
 'Sample Name',
 'Transaction Id',
 'Anon Student Id',
 'Session Id',
 'Time',
 'Time Zone',
 'Duration (sec)',
 'Student Response Type',
 'Student Response Subtype',
 'Tutor Response Type',
 'Tutor Response Subtype',
 'Level (Sequence)',
 'Level (Unit)',
 'Level (Module)',
 'Level (Section1)',
 'Problem Name',
 'Problem View',
 'Problem Start Time',
 'Step Name',
 'Attempt At Step',
 'Is Last Attempt',
 'Outcome',
 'Selection',
 'Action',
 'Input',
 'Feedback Text',
 'Feedback Classification',
 'Help Level',
 'Total Num Hints',
 'KC (Single-KC)',
 'KC Category (Single-KC)',
 'KC (Unique-step)',
 'KC Category (Unique-step)',
 'KC (F2011)',
 'KC Category (F2011)',
 'KC (F2011).1',
 'KC Category (F2011).1',
 'KC (F2011).2',
 'KC Category (F2011).2',
 'School',
 'Class',
 'CF (oli:activityGuid)',
 'CF (oli:highStakes)',
 'CF (oli:purpose)',
 'CF (oli:resourceType)']

In [125]:
for col in cmu_df.columns:
    print(col, cmu_df[col].nunique())

Row 335648
Sample Name 1
Transaction Id 335648
Anon Student Id 228
Session Id 6087
Time 247649
Time Zone 1
Duration (sec) 2425
Student Response Type 5
Student Response Subtype 1
Tutor Response Type 2
Tutor Response Subtype 0
Level (Sequence) 1
Level (Unit) 7
Level (Module) 19
Level (Section1) 9
Problem Name 300
Problem View 32
Problem Start Time 43660
Step Name 382
Attempt At Step 427
Is Last Attempt 2
Outcome 3
Selection 286
Action 9
Input 6451
Feedback Text 1568
Feedback Classification 0
Help Level 0
Total Num Hints 0
KC (Single-KC) 1
KC Category (Single-KC) 0
KC (Unique-step) 1178
KC Category (Unique-step) 0
KC (F2011) 80
KC Category (F2011) 0
KC (F2011).1 18
KC Category (F2011).1 0
KC (F2011).2 8
KC Category (F2011).2 0
School 2
Class 2
CF (oli:activityGuid) 567
CF (oli:highStakes) 2
CF (oli:purpose) 3
CF (oli:resourceType) 2


In [127]:
atmptdf = cmu_df[cmu_df['Outcome'].notnull()]
atmptdf\
[atmptdf['Outcome'].notnull()]\
[atmptdf['Student Response Type'] == 'ATTEMPT']\
.shape

(244715, 46)

In [128]:
atmptdf.Outcome.value_counts()

CORRECT      175858
INCORRECT     68857
HINT          26985
Name: Outcome, dtype: int64